# Baseline Model

In [13]:
%load_ext autoreload
%autoreload 2

import sys
import infostop
import pyproj
    
sys.path.append('./src')
from data_utils import *
clear_output(wait=False)

### Load Train, validation and test data

In [9]:
user_train, user_val, user_test = train, val, test = train_test_data_split()

In [10]:
data_train = pd.concat([create_data_frame(*load_user_data(user), segmentation=True) for user in user_train]).reset_index(drop=True)
data_val = pd.concat([create_data_frame(*load_user_data(user), segmentation=True) for user in user_val]).reset_index(drop=True)
data_test = pd.concat([create_data_frame(*load_user_data(user), segmentation=True) for user in user_test]).reset_index(drop=True)

Segmentation is not strictly needed for the Baseline model, but in order to ensure the datesets accross all three models are the same we still perform it here, and purge all points that would otherwise been purged to build the sequences.

In [5]:
data_train = data_train[data_train['segment_ix'] >= 5]
data_val = data_val[data_val['segment_ix'] >= 5]
data_test = data_test[data_test['segment_ix'] >= 5]

In [14]:
#Projections back and forward from to WGS84 and UTM32
wgs84=pyproj.Proj("+init=EPSG:4326")
etrs89_utm32=pyproj.Proj("+init=EPSG:25832")
project = lambda x, y: pyproj.transform(wgs84, etrs89_utm32, x, y)
inverse = lambda x, y: pyproj.transform(etrs89_utm32, wgs84, x, y)

#### Convert to WGS84

Since this is the native input format of the InfoStop-algoritm

In [ ]:
data_val_wgs84 = np.array([inverse(x[0], x[1]) for x in data_val[['x', 'y']].values])

In [12]:
labels = infostop.label_trace(data_val_wgs84)

AssertionError: Column 0 (latitude) must have values between -90 and 90

### Validation/Test prediction loop

In [ ]:
for phase_name, phase in phases.items():
    if phase['is_training']: 
        continue
    
    phase_dataloader = phase['dataloader']
    phase['predicted'] = []
    phase['true'] = []
    
    for i, batch in enumerate(phase_dataloader, 1):
        X_img, X_tod, y = batch

        phase['true'].extend(list(y.numpy()))
        
        X_img = X_img.to(device, dtype=torch.float)
        X_tod = X_tod.to(device, dtype=torch.float)
        y = y.to(device, dtype=torch.float)
        
        with torch.no_grad():
            out = model(X_img, X_tod)
            probability, predicted = torch.max(out, 1)
            
        phase['predicted'].extend(list(predicted.cpu().numpy()))        

### Final results

In [ ]:
for phase_name, phase in phases.items():
    if phase['is_training']: 
        continue
    
    print(phase_name)
    print(classification_report(phase['true'], phase['predicted'], target_names = ['Motion','Stop']))